# Convolutional Neural Network ( C N N )

# --Version --
# Tensorflow 1.14
# Python 3.7 


In [ ]:
import tensorflow as tf
import random
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np

mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)
print("\n",mnist,"\n")

print(" Train_Data - Image : {} , Label : {}".format(mnist.train.images.shape,mnist.train.labels.shape),"\n"
      ,"Validation_Data - Image : {} , Label : {}".format(mnist.validation.images.shape,mnist.validation.labels.shape),"\n"
      ,"Test_Data - Image : {} , Label : {}\n".format(mnist.test.images.shape,mnist.test.labels.shape))

print("Train_Images_Data_shape : {}".format(mnist.train.images[0].shape))
print(mnist.train.images[0])
print("Train_Label_Data_shape : {}".format(mnist.train.labels[0].shape))
print("Train_Label_Data : {}".format(mnist.train.labels[0]))

In [ ]:
## print(mnist.train.images[0])
a=mnist.train.images[0].reshape(28,28)
plt.imshow(a,cmap='gray')
plt.show()

In [ ]:
#Convolution
sess=tf.InteractiveSession()
img=mnist.train.images[0].reshape(28,28)
img=img.reshape(-1,28,28,1)
w1=tf.Variable(tf.random_normal([3,3,1,5],stddev=0.1))
conv2d=tf.nn.conv2d(img,w1,strides=[1,1,1,1],padding='SAME')
print(conv2d)
sess.run(tf.global_variables_initializer())
conv2d_img=conv2d.eval()
conv2d_img=np.swapaxes(conv2d_img,0,3)
print(conv2d_img)
for i , one_img in enumerate(conv2d_img):
    print(i)
    plt.subplot(1,5,i+1),plt.imshow(one_img.reshape(28,28),cmap='gray')

In [ ]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100

## Build_Model
X = tf.placeholder(tf.float32, [None, 784])
X_img = tf.reshape(X, [-1, 28, 28, 1])
Y = tf.placeholder(tf.float32, [None, 10])


In [ ]:

W1 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev=0.01))#Filter

L1 = tf.nn.conv2d(X_img, W1, strides=[1, 1, 1, 1], padding='SAME')
L2 = tf.nn.relu(L1)
L3 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1],strides=[1, 2, 2, 1], padding='SAME')


In [ ]:

W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01))
L4 = tf.nn.conv2d(L3, W2, strides=[1, 1, 1, 1], padding='SAME')
L5 = tf.nn.relu(L4)
L6 = tf.nn.max_pool(L5, ksize=[1, 2, 2, 1],strides=[1, 2, 2, 1], padding='SAME')
print(L6)
L2_flat = tf.reshape(L6, [-1, 7 * 7 * 64])


In [ ]:

W3 = tf.get_variable("W3", shape=[7 * 7 * 64, 10],initializer=tf.contrib.layers.xavier_initializer())
b = tf.Variable(tf.random_normal([10]))#bias
logits = tf.matmul(L2_flat, W3) + b
# logits = 예측값 ,--- y^ = (X * W) + b
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
            #Gradient_Decent경사하강법
sess = tf.Session()
sess.run(tf.global_variables_initializer())
#########################Build Model

In [ ]:
print(mnist.train.num_examples)
(batch_xs, batch_ys) = mnist.train.next_batch(batch_size)
print(batch_xs.shape)
print(batch_ys.shape)
print(int(mnist.train.num_examples/batch_size))

In [ ]:
print('학습시작')
for epoch in range(training_epochs): #Training_epoch=15
    avg_cost = 0
    total_batch = int(mnist.train.num_examples / batch_size) 
    # mnist.train.num_examples = 55000
    # batch_size = 100
    # total_batch = 550
    for i in range(total_batch):
        batch_xs,batch_ys = mnist.train.next_batch(batch_size)
        feed_dict = {X: batch_xs, Y: batch_ys}
        c,_ = sess.run([cost, optimizer], feed_dict=feed_dict)
        avg_cost += c / total_batch
    print('Epoch : {}'.format(epoch + 1), '에폭cost = {:.9f}'.format(avg_cost))
print('학습 끝')

In [ ]:
##############################테스트
# test=tf.equal(2,1)
# print(sess.run(test))
# test2=tf.reduce_mean(tf.cast(test,tf.float32))
# print(sess.run(test2))
##############################테스트

In [ ]:
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1)) #True or False
#[[3 10 1]       if 0== 1 0 2
# [4 5 6]        if 1==1 2 1
# [0 8 7]]
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32)) #True면 1 False 면 0
# print(sess.run(correct_prediction, feed_dict={X: mnist.test.images, Y: mnist.test.labels}))
print('Accuracy:', sess.run(accuracy, feed_dict={X: mnist.test.images, Y: mnist.test.labels}))

In [ ]:
#Test
for i in range(20):
    r = random.randint(0, mnist.test.num_examples - 1)
    print("Label: ", np.argmax(mnist.test.labels[r]))
    print("Prediction: ", sess.run(tf.argmax(logits, 1), feed_dict={X: mnist.test.images[r:r + 1]}))
                                                            #Input_Demention = [N,784]
                                                            #mnist.test.images[r:r+1].shape = [1,784]
                                                            #mnist.test.images[r].shape = [ ,784]
    plt.imshow(mnist.test.images[r].reshape(28, 28), cmap='gray')
    plt.show()